https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist.py  
Builds the MNIST network.  
1. inference() - Builds the model as far as is required for running the network forward to make predictions.
2. loss() - Adds to the inference model the layers required to generate loss.
3. training() - Adds to the loss model the Ops required to generate and apply gradients.

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import tensorflow as tf

In [7]:
NUM_CLASSES = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE

In [50]:
def ｉｎference(images, hidden1_units, hidden2_units):
    """
    hiden1_units: size of first hidden layer
    """
    # hidden 1
    with tf.name_scope('hidden1'):
        weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, hidden1_units],
                              stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))))
        biases = tf.Variable(tf.zeros([hidden1_units]))
        hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)

    #hidden 2
    with tf.name_scope('hidden2'):
        weights = tf.Variable(tf.truncated_normal([hidden1_units, hidden2_units],
                              stddev=1.0 / math.sqrt(float(hidden1_units))))
        biases = tf.Variable(tf.zeros([hidden2_units]))
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)

    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(tf.truncated_normal([hidden2_units, NUM_CLASSES],
                              stddev=1.0 / math.sqrt(float(hidden2_units))))
        biases = tf.Variable(tf.zeros([NUM_CLASSES]))
        logits = tf.matmul(hidden2, weights) + biases
    
    return logits #logits.shape　[batchsize, num_classes]

In [51]:
def loss(logits, labels):
    """Calculates the loss from the logits and the labels.
      Args:
          logits: Logits tensor, float - [batch_size, NUM_CLASSES].
          labels: Labels tensor, int32 - [batch_size].
      Returns:
          loss: Loss tensor of type float.
      """
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)
    #ｔｆ.nn.sparse_softmax_cross_entropy_with_logits可以自动将ｌａｂｅｌｓ转成１－ｈｏｔ编码,然后计算
    return tf.reduce_mean(cross_entropy) # renturn loss

In [52]:
def training(loss, learning_rate):
    """
    return train operation
    """
    # 写ｓｕｍｍａｒｙ
    tf.summary.scalar('loss', loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    global_step = tf.Variable(0, trainable=False) # a variable to track the global step
    train_op = optimizer.minimize(loss, global_step = global_step) # global_step is a counter
    return train_op

In [53]:
def evaluation(logits, labels):
    # It returns a bool tensor with shape [batch_size]
    correct = tf.nn.in_top_k(logits, labels, k = 1)
    return tf.reduce_sum(tf.cast(correct, tf.int32))

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/fully_connected_feed.py  
Trains and Evaluates the MNIST network using a feed dictionary.

In [54]:
import argparse
import os.path
import sys
import time

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

# Basic model parameters as external flags.
FLAGS = None

In [55]:
def placeholder_inputs(batch_size):
    images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
    return images_placeholder, labels_placeholder

In [56]:
def fill_feed_dict(data_set, images_pl, labels_pl):
    images_feed, labels_feed = data_set.next_batch(FLAGS.batch_size, FLAGS.fake_data)
    feed_dict = {images_pl: images_feed, labels_pl: labels_feed}
    return feed_dict

In [57]:
def do_eval(sess, eval_correct, image_placeholder, label_placeholder, data_set):
    true_count = 0 # counts #correct predictions
    steps_per_epoch = data_set.num_examples // FLAGS.batch_size
    num_examples = steps_per_epoch * FLAGS.batch_size
    
    for step in xrange(steps_per_epoch):
        feed_dict = fill_feed_dict(data_set, image_placeholder, label_placeholder)
        true_count += sess.run(eval_correct, feed_dict = feed_dict)
        precision = float(true_count) / num_examples
        print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
              (num_examples, true_count, precision))

In [64]:
def run_training():
    data_sets = input_data.read_data_sets(FLAGS.input_data_dir, FLAGS.fake_data)
    
    # tell tf that the model will be built into the default graph
    with tf.Graph().as_default():
        images_placeholder, labels_placeholder = placeholder_inputs(FLAGS.batch_size)

        logits = inference(images_placeholder, FLAGS.hidden1, FLAGS.hidden2)
        loss_ = loss(logits, labels_placeholder)
        train_op = training(loss_, FLAGS.learning_rate)
        eval_correct = evaluation(logits, labels_placeholder)
        
        # Build the summary Tensor based on the TF collection of Summaries.
        summary = tf.summary.merge_all()
        
        init = tf.global_variables_initializer()
        
        # create a saver for writing training checkpoints
        saver = tf.train.Saver()
        
        sess = tf.Session()
        
        # Instantiate a SummaryWriter to output summaries and the Graph.
        summary_writer = tf.summary.FileWriter(FLAGS.log_dir, sess.graph)
        
        sess.run(init)
        
        for step in xrange(FLAGS.max_steps):
            start_time = time.time()
            feed_dict = fill_feed_dict(data_sets.train, images_placeholder, labels_placeholder)
            
            _, loss_value = sess.run([train_op, loss_], feed_dict = feed_dict)
            duration = time.time() - start_time
            
            if step % 100 == 0:
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                summary_str = sess.run(summary, feed_dict=feed_dict)
                summary_writer.add_summary(summary_str, step)
                summary_writer.flush()

In [65]:
def main(_):
    if tf.gfile.Exists(FLAGS.log_dir):
        tf.gfile.DeleteRecursively(FLAGS.log_dir)
    tf.gfile.MakeDirs(FLAGS.log_dir)
    run_training()

In [67]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--learning_rate',
        type=float,
        default=0.01,
        help='Initial learning rate.'
    )
    parser.add_argument(
        '--max_steps',
        type=int,
        default=2000,
        help='Number of steps to run trainer.'
    )
    parser.add_argument(
        '--hidden1',
        type=int,
        default=128,
        help='Number of units in hidden layer 1.'
    )
    parser.add_argument(
        '--hidden2',
        type=int,
        default=32,
        help='Number of units in hidden layer 2.'
    )
    parser.add_argument(
        '--batch_size',
        type=int,
        default=100,
        help='Batch size.  Must divide evenly into the dataset sizes.'
    )
    parser.add_argument(
        '--input_data_dir',
        type=str,
        default='/tmp/tensorflow/mnist/input_data',
        help='Directory to put the input data.'
    )
    parser.add_argument(
        '--log_dir',
        type=str,
        default='/tmp/tensorflow/mnist/logs/fully_connected_feed',
        help='Directory to put the log data.'
    )
    parser.add_argument(
        '--fake_data',
        default=False,
        help='If true, uses fake data for unit testing.',
        action='store_true'
    )

    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
Step 0: loss = 2.31 (0.013 sec)
Step 100: loss = 2.14 (0.004 sec)
Step 200: loss = 1.86 (0.003 sec)
Step 300: loss = 1.57 (0.003 sec)
Step 400: loss = 1.32 (0.002 sec)
Step 500: loss = 0.84 (0.003 sec)
Step 600: loss = 0.80 (0.003 sec)
Step 700: loss = 0.60 (0.003 sec)
Step 800: loss = 0.63 (0.002 sec)
Step 900: loss = 0.57 (0.003 sec)
Step 1000: loss = 0.48 (0.003 sec)
Step 1100: loss = 0.56 (0.051 sec)
Step 1200: loss = 0.43 (0.004 sec)
Step 1300: loss = 0.37 (0.003 sec)
Step 1400: loss = 0.35 (0.003 sec)
Step 1500: loss = 0.45 (0.003 sec)
Step 1600: loss = 0.37 (0.003 sec)
Step 1700: loss = 0.45 (0.003 sec)
Step 1800: loss = 0.36 (0.003 sec)
Step 1900: loss = 0.38 (0.003 sec)


SystemExit: 

/home/cis/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [68]:
%tb

SystemExit: 